In [1]:
import tensorflow as tf

from fuel.datasets import MNIST
mnist = MNIST(('train',))
state = mnist.open()

In [2]:
class FClayer():
    def __init__(self,shape):
        self.weights = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
        self.biases = tf.Variable(tf.zeros([10])+1)
        
    def forward(self,x):
        return tf.nn.softmax(tf.matmul(x,self.weights) + self.biases)
    


In [3]:
x = tf.placeholder("float", [None, 784])

layer1 = FClayer([784,10])
y = layer1.forward(x)





In [4]:
y_ = tf.placeholder("float", [None,10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [5]:
import numpy as np
def onehot(labels_dense, num_classes=10):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = np.arange(num_labels) * num_classes
  labels_one_hot = np.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

In [6]:
n=50
#A method to give us some random batch samples
from fuel.schemes import ShuffledScheme 
scheme = ShuffledScheme(examples=int(mnist.num_examples), batch_size=n)

#For each batch
for i,request in enumerate(scheme.get_request_iterator()):
    im,labels = mnist.get_data(state=state, request=request)
    sess.run(train_step, feed_dict={x: im.reshape(n,28*28)/255, y_: onehot(labels)})
    if i%(20) == 1:
        
        acc,CE,Y = sess.run([accuracy,cross_entropy,y], feed_dict= {x: im.reshape(n,28*28)/255, y_:  onehot(labels)})
        print("step %d, training accuracy %g"%(i, acc) )

mnist = MNIST(('test',))
state = mnist.open()
#all of the test data
req = [n for n in range(mnist.num_examples)] 
im,labels = mnist.get_data(state=state, request=req)

acc_val = sess.run(accuracy, feed_dict={x: im.reshape(mnist.num_examples,28*28)/255, y_:  onehot(labels)})
print('Accuracy = '+str(acc_val) + '%')

step 1, training accuracy 0.5
step 21, training accuracy 0.9
step 41, training accuracy 0.92
step 61, training accuracy 0.86
step 81, training accuracy 0.9
step 101, training accuracy 0.94
step 121, training accuracy 0.92
step 141, training accuracy 0.9
step 161, training accuracy 0.9
step 181, training accuracy 0.92
step 201, training accuracy 0.94
step 221, training accuracy 0.98
step 241, training accuracy 0.92
step 261, training accuracy 0.9
step 281, training accuracy 0.9
step 301, training accuracy 0.94
step 321, training accuracy 0.94
step 341, training accuracy 0.88
step 361, training accuracy 0.94
step 381, training accuracy 0.92
step 401, training accuracy 0.94
step 421, training accuracy 0.96
step 441, training accuracy 1
step 461, training accuracy 0.94
step 481, training accuracy 0.98
step 501, training accuracy 0.96
step 521, training accuracy 0.94
step 541, training accuracy 0.92
step 561, training accuracy 0.94
step 581, training accuracy 1
step 601, training accuracy 0